In [1]:
import numpy as np
import pandas as pd

In [2]:
samples = np.random.multivariate_normal([3,4,5,6],np.diag([1,2,3,4]),size=50000)
w = 10000

In [3]:
def emp_mu(window):
    return pd.DataFrame(window).mean().values

def emp_cov(window):
    mu = emp_mu(window)
    cov = np.diag([0,0,0,0])
    
    for i in range(window.shape[0]):
        diff = window[i,:] - mu
        cov = cov + (1/w)*diff.reshape(-1,1).dot(diff.reshape(-1,1).T)
        
    return cov

def emp_cov_2(window):
    mu = emp_mu(window)
    cov = -mu.reshape(-1,1).dot(mu.reshape(-1,1).T)
    
    for i in range(window.shape[0]):
        cov = cov + (1/w)*window[i,:].reshape(-1,1).dot(window[i,:].reshape(-1,1).T)
        
    return cov
    
t = 11000
window_t = samples[t-w:t,:]
window_tp1 = samples[t-w+1:t+1,:]

In [4]:
pd.DataFrame(emp_cov(window_t))

,0,1,2,3
0,1.006396,0.007550,-0.006021,0.005707
1,0.007550,2.018216,-0.002047,-0.000144
2,-0.006021,-0.002047,2.972636,-0.021646
3,0.005707,-0.000144,-0.021646,3.924767


In [17]:
pd.DataFrame(emp_cov(window_tp1))

,0,1,2,3
0,1.006278,0.007577,-0.006134,0.005912
1,0.007577,2.018278,-0.001904,-0.000364
2,-0.006134,-0.001904,2.972726,-0.021747
3,0.005912,-0.000364,-0.021747,3.924855


In [16]:
def my_cov_diff(t):
    add_in = (1/w)*samples[t,:].reshape(-1,1).dot(samples[t,:].reshape(-1,1).T) 
    subtract_out = (1/w)*samples[t-w,:].reshape(-1,1).dot(samples[t-w,:].reshape(-1,1).T)
    
    diff = add_in - subtract_out
    
    delta = (1/w)*(samples[t,:] - samples[t-w,:])
    Ert = emp_mu(samples[t-w:t,:])
    
    ch= Ert.reshape(-1,1).dot(delta.reshape(-1,1).T) - delta.reshape(-1,1).dot(Ert.reshape(-1,1).T)
    print(ch)
    
    diff = diff - Ert.reshape(-1,1).dot(delta.reshape(-1,1).T)
    diff = diff - delta.reshape(-1,1).dot(Ert.reshape(-1,1).T)
    diff = diff - delta.reshape(-1,1).dot(delta.reshape(-1,1).T)
    
    return diff

def my_cov_diff_2(t):
    
    Winv = np.diag([1/w,-1/w,-1/w,-1/w,-1/(w**2)])
    
    delta = samples[t,:] - samples[t-w,:]
    mu = emp_mu(samples[t-w:t,:])
    
    L = np.column_stack([samples[t,:], 
                          samples[t-w,:],
                          mu,
                          delta,
                          delta])
    R = np.column_stack([samples[t,:], 
                          samples[t-w,:],
                          delta,
                          mu,
                          delta])
    
    return L.dot(Winv).dot(R.T)

pd.DataFrame(my_cov_diff_2(t) +   emp_cov(window_t))   

,0,1,2,3
0,1.006278,0.007577,-0.006134,0.005912
1,0.007577,2.018278,-0.001904,-0.000364
2,-0.006134,-0.001904,2.972726,-0.021747
3,0.005912,-0.000364,-0.021747,3.924855
